# Quick Start

本章節將帶你快速走過一個 **LlamaIndex 向量檢索（Vector-based RAG）** 的最小可行流程（Minimal Working Example），包含：

1. 載入 Embedding Model  
2. 建立 Vector Store  
3. 將文件切分為 Nodes 並建立 Embeddings
    - Node 是 LlamaIndex 甚至整個 RAG 架構中，檢索（Retrieval）的「最小原子單位」。
4. 建立 Index
    - Index 是連接 Node 和數據存儲的邏輯橋樑。
    - 組織者：它決定了數據如何被組織。最常見的是 VectorStoreIndex，它會把 Node 的向量存入 Vector Store。
    - 中介：當你進行查詢時，你是向 Index 提問，Index 會去調度 Vector Store 尋找最匹配的 Node，並將其回傳。
6. 使用 Retriever 進行查詢  

此流程是建構 **Retrieval-Augmented Generation（RAG）系統** 的基礎。

## Ollama Cloud Service with LlamaIndex

In [ ]:
import os

os.chdir("../")

from initialization import credential_init

In [ ]:
from llama_index.llms.ollama import Ollama

model = "gpt-oss:120b-cloud"
base_url = "https://ollama.com"

credential_init()

ollama_llm = Ollama(model=model, request_timeout=60.0, 
                         base_url=base_url)

也可以從 src.ollama_connection 裡直接調用:

```
from src.ollama_connection import llama_index_ollama

ollama_llm = llama_index_ollama(model=model, temperature=0)
```

In [ ]:
response = await ollama_llm.acomplete("What is the capital of France?")

## Loading embedding model

在 RAG 系統中，**Embedding Model** 負責將文字轉換成高維度向量（vector），以便後續進行語意相似度搜尋。

這裡我們使用 HuggingFace 上的 **BAAI/bge-small-en-v1.5**：
- 輕量、速度快
- 適合本地端或教學示範
- 向量維度為 **384**

```python
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
```

📌 注意
Embedding model 的輸出維度，必須與後面 Vector Store 使用的向量維度一致，否則會產生錯誤。

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads https://huggingface.co/BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

## Loading a vector store

Vector Store 用來儲存：

>- Node 的向量表示（embeddings）

>- 對應的 Node ID

這裡使用 FAISS（Facebook AI Similarity Search）：

>- 高效能

>- 常見於本地向量檢索場景

In [ ]:
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore

d = 384 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)

📌 IndexFlatL2
表示使用 L2 距離（歐式距離）來計算向量相似度。

## Creating Nodes and Their Embeddings

為什麼需要 Nodes？

在 LlamaIndex 中：

>- Document 是原始文件
>- Node 是可被檢索的最小單位

Node Parser 決定了：

>- 文件如何被切分
>- 每個 Node 的上下文範圍與 metadata

---
SentenceWindowNodeParser（句子視窗解析器）

此 Parser 會：

>- 將文件切成「單一句子」
>- 但在 metadata 中保留前後的「句子視窗（window）」

這對 embedding 非常有用：

>- embedding 很精準（單句）
>- 回答時仍保有上下文（window）

---
IngestionPipeline（資料處理管線）

IngestionPipeline 將多個步驟串在一起：

>1. Node Parser（文件 → Nodes）
>2. Embedding Model（Nodes → Vectors）
>3. 寫入 Vector Store

In [ ]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter, SentenceWindowNodeParser
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

node_parser = SentenceWindowNodeParser(
    # How many sentences on both sides to capture. 
    # Setting this to 3 results in 7 sentences.
    window_size=3, # # 前後各 3 句，共 7 句
    # the metadata key for to be used in MetadataReplacementPostProcessor
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence"
)


# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        node_parser,
        embed_model,
    ],
    vector_store=vector_store,
)

# run the pipeline
nodes = pipeline.run(documents=[Document.example()])

In [ ]:
len(nodes)

📌 好處

>- 流程模組化
>- 易於替換 parser / embedding / extractor
>- 適合正式專案使用

In [ ]:
nodes[0].metadata

## Building the Index（建立索引）

Index 是：

>- Vector Store 的抽象封裝
>- 提供 Retriever 與 Query Engine 的入口

Storage Context 的核心功用

在 LlamaIndex 中，資料不只是「向量」而已。一個完整的 RAG 系統需要儲存多種資訊，Storage Context 負責統一管理以下幾個組件：

- Vector Store (向量儲存)：
    - 儲存 Node 的 Embedding（向量）。
    - 用於「語義搜尋」。

- Docstore (文件儲存)：
    - 儲存 Node 的 原始文本內容 和 元數據 (Metadata)。
    - 為什麼需要？ 因為向量數據庫有時只存向量，當搜尋到匹配的向量後，系統需要回頭從 Docstore 抓取真正的文字給 LLM 看。

- Index Store (索引儲存)：

    - 儲存 Index 本身的結構資訊（例如：這棵樹是怎麼連的，或者這個摘要索引的層級關係）。

- Property Graph Store (屬性圖儲存)：

    - （選配）用於儲存知識圖譜（Knowledge Graph）的實體與關係。
 
![caption](Gemini_Generated_Image_gfpi58gfpi58gfpi.png)

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex

storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    llm=ollama_llm
)

index = VectorStoreIndex(nodes, embed_model=embed_model)

## Retrieving Nodes（語意檢索）

Retriever 負責：

>- 將 query 轉成 embedding
>- 與向量資料庫比對
>- 回傳最相似的 Nodes

In [ ]:
retriever = index.as_retriever(similarity_top_k=3)

In [ ]:
retriever.retrieve("what is LLM?")

回傳結果是 Node list，不是最終文字答案

這些 Nodes 會在後續交給 LLM 產生回應。

In [ ]:
# Create vector index from base nodes
# base_index = VectorStoreIndex(base_nodes)

# Instantiate query engine from vector index
base_query_engine = index.as_query_engine(
    similarity_top_k=1,
)

# Send query to the engine to get related node(s)
base_response = base_query_engine.query("what is LLM?")

print(base_response)

In [ ]:
base_response.source_nodes

# Basic Node Parsers

官方文檔: https://developers.llamaindex.ai/python/framework/module_guides/loading/node_parsers/modules/

在 **LlamaIndex** 中，**Node Parser**（節點解析器）負責將原始文件（如 PDF、Markdown、純文字）切分成較小、結構化的單位，稱為 **Nodes**。這些 Nodes 是後續建立索引、檢索與語意搜尋的基礎。

## 為什麼需要 Node Parser？
- 🧩 **提升檢索品質**：適當的切分可讓模型更精準理解上下文  
- ⚖️ **控制上下文長度**：避免單次輸入過長，超出 LLM 限制  
- 🏗️ **保留結構資訊**：可包含 metadata（如段落、頁碼、標題）

## 常見的 Node Parser 類型
- **SentenceSplitter**：依句子切分（最常用）
- **TokenTextSplitter**：依 token 數量切分，適合精細控制
- **MarkdownNodeParser**：保留 Markdown 結構（如標題、清單）
- **HierarchicalNodeParser**：建立父子階層的節點結構

## 基本使用範例
```python
from llama_index.node_parser import SentenceSplitter

parser = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=50
)

nodes = parser.get_nodes_from_documents(documents)
```

## 總結

Node Parser 是 LlamaIndex 文件處理流程中的關鍵元件，決定了資料如何被切分與理解。選擇合適的 Parser 與參數，能顯著影響 RAG（Retrieval-Augmented Generation）系統的效能與準確度。

## 基於檔案的Node Parser

有多種**以檔案為基礎的 Node Parser**，會根據被解析的內容類型（例如 JSON、Markdown 等）來建立對應的節點（Nodes）。

最簡單的流程是將 **FlatFileReader** 與 **SimpleFileNodeParser** 結合使用，系統會自動為每一種內容類型選擇最合適的 Node Parser。接著，你可以再將這類檔案型 Node Parser 與**文字型 Node Parser** 串接使用，以進一步考量實際文字長度，確保切分結果更符合後續處理需求。


### SimpleFileNodeParser

In [ ]:
from pathlib import Path

from llama_index.core.node_parser import SimpleFileNodeParser
from llama_index.readers.file import FlatReader


md_docs = FlatReader().load_data(Path("week_1/week-1.md"))

parser = SimpleFileNodeParser()
md_nodes = parser.get_nodes_from_documents(md_docs)

### HTMLNodeParser
此 Node Parser 會使用 **BeautifulSoup** 來解析原始的 HTML 內容。

預設情況下，它只會解析一部分指定的 HTML 標籤，但你也可以自行覆寫這個設定。

預設解析的標籤包含：
`["p", "h1", "h2", "h3", "h4", "h5", "h6", "li", "b", "i", "u", "section"]`

可執行以下指令來取得範例 HTML 檔案：

```bash
python -m html_file_demo

In [ ]:
from llama_index.core.node_parser import HTMLNodeParser

html_docs = FlatReader().load_data(Path("week_1/Titan.html"))

parser = HTMLNodeParser(tags=["p", "h1", 'u', 'section'])  # optional list of tags
nodes = parser.get_nodes_from_documents(html_docs)

### JSONNodeParser

用於解析原始的 JSON 資料。

In [ ]:
from llama_index.core.node_parser import JSONNodeParser

json_docs = FlatReader().load_data(Path("week_1/mock_data.json"))

parser = JSONNodeParser()

nodes = parser.get_nodes_from_documents(json_docs)

### MarkdownNodeParser
用於解析原始的 Markdown 文字內容。

In [ ]:
from pathlib import Path

from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import MarkdownNodeParser

parser = MarkdownNodeParser()

md_docs = FlatReader().load_data(Path("week_1/week-1.md"))

nodes = parser.get_nodes_from_documents(md_docs)

## Text-Splitters

不是越大越好

>- Chunk 太大 → embedding 混合多個主題，相似度失準
>- Chunk 太小 → 語意破碎，LLM 回答品質下降

### CodeSplitter

會根據程式碼所使用的程式語言，將原始的程式碼文字進行切分。

可在以下連結查看完整支援的程式語言清單：

https://github.com/grantjenks/py-tree-sitter-languages#license

In [ ]:
from pathlib import Path

from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import CodeSplitter

documents = FlatReader().load_data(Path("week_1/mock_python.py"))

splitter = CodeSplitter(
    language="python",
    chunk_lines=40,  # lines per chunk
    chunk_lines_overlap=15,  # lines overlap between chunks
    max_chars=1500,  # max chars per chunk
)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
print(nodes[0].text)

In [ ]:
print(nodes[1].text)

### LangchainNodeParser
你也可以將 **LangChain** 中現有的任何文字切分器（text splitter）包裝成一個 Node Parser 來使用。

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import LangchainNodeParser

documents = FlatReader().load_data(Path("week_1/mock_langchain_text.md"))

parser = LangchainNodeParser(RecursiveCharacterTextSplitter(chunk_size=250))
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
print(nodes[0].text)

In [ ]:
print(nodes[2].text)

### SentenceSplitter
**SentenceSplitter** 會在盡量保留句子邊界的前提下，對文字內容進行切分。

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

documents = FlatReader().load_data(Path("week_1/mock_langchain_text.md"))

sentence_splitter_node_parser = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = sentence_splitter_node_parser.get_nodes_from_documents(documents)

In [ ]:
splitter.split_text

In [ ]:
# ToDo 移去 Advance Node Parser
# ### SentenceWindowNodeParser

# `SentenceWindowNodeParser` 與其他節點解析器相似，不過它會將所有文件拆分成單獨的句子。  
# 生成的節點在其 metadata 中還包含了每個節點周圍的「句子窗口」。  
# > ⚠️ 請注意，這些 metadata 不會被 LLM 或嵌入模型看到。  

# 這在生成具有非常特定範圍的嵌入時最為有用。  
# 然後，結合 `MetadataReplacementNodePostProcessor`，您可以在將節點傳送給 LLM 之前，用其周圍的上下文替換該句子。  

# 下面是一個使用預設設定來設置解析器的範例：  
# > 在實際應用中，您通常只需要調整句子的窗口大小即可。


In [ ]:
# from llama_index.core.node_parser import SentenceWindowNodeParser

# node_parser = SentenceWindowNodeParser.from_defaults(
#     # how many sentences on either side to capture
#     window_size=3,
#     # the metadata key that holds the window of surrounding sentences
#     window_metadata_key="window",
#     # the metadata key that holds the original sentence
#     original_text_metadata_key="original_sentence",

### SemanticSplitterNodeParser

SemanticSplitter 不是負責斷句的，
它只是在「已斷好的句子」之間找語意跳躍。

In [ ]:
from llama_index.core.node_parser import SemanticSplitterNodeParser

documents = FlatReader().load_data(Path("week_1/semantic_test_zh.txt"))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=32,
    chunk_overlap=0,
    separators=["\n\n", "\n", "。", "，", " "]
).split_text

# text_splitter = SentenceSplitter(chunk_size=64,
#                                  chunk_overlap=0).split_text

semantic_splitter_node_parser = SemanticSplitterNodeParser.from_defaults(
    embed_model = embed_model,
    sentence_splitter=text_splitter,
    include_metadata=True,
    include_prev_next_rel=True,
    breakpoint_percentile_threshold=75,
)

nodes = semantic_splitter_node_parser.get_nodes_from_documents(documents)

In [ ]:
nodes[0]

In [ ]:
nodes[1]

In [ ]:
nodes[2]

### TokenTextSplitter
`TokenTextSplitter` 嘗試根據原始的 **token 數量** 將文本切分成大小一致的塊（chunk）。

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter

node_parser = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)

# VectorStore

這裡我們展示如何宣告vectorstore，然後在Index的部分示範如何retrieve。

## FAISS

In [ ]:
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore

d = 1024 # 必須與 embedding model 的輸出維度一致，這邊配合 BAAI/bge-m3

faiss_index = faiss.IndexFlatL2(d)
faiss_vector_store = FaissVectorStore(faiss_index=faiss_index)

## Qdrant

In [ ]:
from qdrant_client import AsyncQdrantClient
from llama_index.vector_stores.qdrant import QdrantVectorStore

client = AsyncQdrantClient(path="week_1/langchain_qdrant")

qdrant_vector_store = QdrantVectorStore(
    client=client,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

# Index - Basic

## VectorIndexStore

建立documents。先聲明我沒買他們股票，也不提供投資建議

In [ ]:
from pathlib import Path

from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(
    chunk_size=1000,
    chunk_overlap=120,
)

loader = PyMuPDFReader()

files = [
    "week_1/TSMC_2024_年度財報.pdf",
    "week_1/富邦金_2024_年度財報.pdf",
]

docs = []
for f in files:
    doc_text = "\n\n".join([d.get_content() for d in loader.load(f)])
    docs.append(Document(text=doc_text, metadata={"file_path":f}))

In [ ]:
docs[0].metadata

In [ ]:
docs[0].id_

### 搭配 FAISS

這裡的做法算是一種快速暴力的手段。務實上會用

parser → nodes → index

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=faiss_vector_store)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

faiss_index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context,
    embed_model=embed_model,
    llm=ollama_llm,
    transformations=[text_splitter],
)

In [ ]:
faiss_retriever = faiss_index.as_retriever(similarity_top_k=3)

In [ ]:
output = await faiss_retriever.aretrieve("年度營業額")

In [ ]:
output

**FAISS 不支援在metadata層面過濾節點**

In [ ]:
from llama_index.core.vector_stores import MetadataFilters, ExactMatchFilter

output = await faiss_retriever.aretrieve("富邦年度營業額")

In [ ]:
output

### 搭配 Qdrant

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=qdrant_vector_store)

qdrant_index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context,
    embed_model=embed_model,
    llm=ollama_llm,
    transformations=[text_splitter],
    use_async=True # 因為我們使用async client
)

```
ValueError: Async client is not initialized!
Please pass in `aclient` to the constructor: `QdrantVectorStore(..., aclient=AsyncQdrantClient(...))`
Selection deleted
```


In [ ]:
qdrant_retriever = qdrant_index.as_retriever(similarity_top_k=3)

output = await qdrant_retriever.aretrieve("年度營業額")

In [ ]:
output

透過metadata進行過濾

In [ ]:
from llama_index.core.vector_stores import (
    MetadataFilters,
    ExactMatchFilter,
)

filters = MetadataFilters(
    filters=[
        ExactMatchFilter(key="file_path", value="week_1/富邦金_2024_年度財報.pdf"),
        # ExactMatchFilter(key="source", value="manual.pdf"),
    ]
)

qdrant_retriever = qdrant_index.as_retriever(
    similarity_top_k=3,
    filters=filters,
)

In [ ]:
output = qdrant_retriever.retrieve("年度營業額")

In [ ]:
output

# 評估 (Evaluation)

我們這裡用LlamaIndex提供的Metrics來示範如何進行評估
後面我們會使用RAGAS這個框架進行評估

> ⚠️ 注意：這個過程可能花費較高成本。請小心使用，並根據預算調整樣本數量。

產生nodes

In [ ]:
nodes = text_splitter.get_nodes_from_documents(docs)

In [ ]:
len(nodes)

In [ ]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
import nest_asyncio

"""
nest_asyncio 會 打補丁（patch） 現有的事件循環，使得 在已經運行的事件循環中可以再次嵌套運行 asyncio。

換句話說，它讓你在 Jupyter Notebook 裡可以順利執行：

asyncio.run()

自己寫的 async 函數

各種需要事件循環的庫（例如 LLM SDK、WebSocket、FastAPI 的測試等）

而不會碰到「事件循環已經運行」的錯誤。
"""
nest_asyncio.apply()

## 人工數據合成

`num_questions_per_chunk` 與 `num` 的差異說明

在使用 DatasetGenerator（例如 `generate_dataset_from_nodes`）時，  
`num_questions_per_chunk` 與 `num` 分別控制 **深度** 與 **廣度**。

---

## 1. `num_questions_per_chunk`（每個 Node 產生的問題數量）

### 定義
此參數定義 LLM 在處理**單一 Node（chunk）**時，  
要從該 Node 的內容中挖掘出多少個問題。

- **作用對象**：單一 Node
- **控制面向**：深度（depth）

### 行為邏輯
- 設為 `1`  
  → LLM 讀取 Node A，產生 **1 個問題**
- 設為 `3`  
  → LLM 針對 Node A 的內容，產生 **3 個不同的問題**

### 影響
- 數值越高：
  - 對單一內容的測試越深入、越嚴密
- 但如果：
  - Node 本身內容較少
  - 或資訊密度不高  
  則可能導致：
  - 問題重複
  - 或問題品質下降

---

## 2. `num`（總共處理的 Node 數量）

### 定義
此參數定義 DatasetGenerator  
**總共要從 `selected_nodes` 中挑選多少個 Node 來生成問題**。

- **作用對象**：整個 Node 列表
- **控制面向**：廣度（breadth）

### 行為邏輯
- 假設你有 `100` 個 Nodes
- 若設定 `num = 5`
  - 只會選取前 `5` 個 Nodes 來生成問題

### 影響
- 決定測試資料集涵蓋：
  - 多少原始文件內容
  - 多少主題範圍

---

## 3. 題目總數的計算方式

你可以用以下公式來估算最終生成的題目數量：

$$
\text{Total Questions} = \text{num} \times \text{num\_questions\_per\_chunk}
$$

### 範例
```python
num = 5
num_questions_per_chunk = 1
```

- 處理 5 個 Nodes
- 每個 Node 產生 1 題

👉 最終產出：5 個問題

---

## 4. 直覺圖解（白話比喻）

可以把這兩個參數想像成「出考題」的方式：

- num：選幾個人
    從全班中選出 5 個學生
- num_questions_per_chunk：每人寫幾題
    要求這 5 個學生

每人各寫 1 題考卷

👉 最後你就會拿到 5 題考題

## 5. 總結

| 參數                        | 控制面向 | 問的是什麼         |
| ------------------------- | ---- | ------------- |
| `num`                     | 廣度   | 要涵蓋多少 Nodes？  |
| `num_questions_per_chunk` | 深度   | 每個 Node 要挖多深？ |


In [ ]:
import numpy as np

selected_nodes = np.random.choice(nodes, size=50, replace=False)

可以透過`text_question_template`這個參數來自定義數據生成使用的prompt。盡量保持格式一致，另外別忘了{context_str}和{query_str}是必備的。

In [ ]:
from llama_index.core import PromptTemplate

# 1. 將字串封裝成 PromptTemplate 物件
text_question_template = PromptTemplate(
    'Context information is below.\n'
    '---------------------\n'
    '{context_str}\n'
    '---------------------\n'
    'Given the context information and not prior knowledge.\n'
    'generate only questions based on the below query.\n'
    '{query_str}\n'
    '。所有的輸出必須使用繁體中文(traditional Chinese)'
)

一次扔50個進去 Ollama Cloud會抱怨

In [ ]:
import asyncio

# 分批生成，例如一次只處理 5 個 node，每批之間休息一下
async def batch_generate_questions(all_nodes, batch_size=5):
    all_questions = []
    # 將 50 個 nodes 分成每 5 個一組
    for i in range(0, len(all_nodes), batch_size):
        batch_nodes = all_nodes[i:i + batch_size]
        print(f"正在處理第 {i} 到 {i+batch_size} 個節點...")

        dataset_generator = DatasetGenerator(
            batch_nodes,
            llm=ollama_llm,
            text_question_template=text_question_template,
            num_questions_per_chunk=1,
        )
        
        # 針對這一小批生成問題
        # 注意這裡 num 就等於 batch_size
        batch_questions = await dataset_generator.agenerate_dataset_from_nodes(num=len(batch_nodes))
        all_questions.append(batch_questions)
        
        # 讓 Ollama 喘口氣，休息 2 秒
        await asyncio.sleep(2) 
    
    return all_questions

eval_dataset = await batch_generate_questions(all_nodes=selected_nodes[:10], batch_size=2)

In [ ]:
len(eval_dataset)

In [ ]:
from llama_index.core.evaluation.dataset_generation import  QueryResponseDataset

combined_queries = {}
combined_responses = {}

for ds in eval_dataset:
    # 將每個 dataset 的字典合併到總字典中
    combined_queries.update(ds.queries)
    combined_responses.update(ds.responses)
    
# 回傳一個全新的 Dataset 物件
final_dataset = QueryResponseDataset(
    queries=combined_queries,
    responses=combined_responses
)

In [ ]:
final_dataset.qr_pairs

## Metrics

| Evaluator          | 評估重點    | 是否需要參考答案 | 是否使用 Context |
| ------------------ | ------- | -------- | ------------ |
| Correctness        | 是否正確    | 需要       | 不一定          |
| SemanticSimilarity | 語意是否相似  | 需要       | 不一定          |
| Relevancy          | 是否回答問題  | 不需要      | 不需要          |
| Faithfulness       | 是否忠實於資料 | 不需要      | 需要           |

In [ ]:
from collections import defaultdict

import pandas as pd
from llama_index.core.evaluation import CorrectnessEvaluator, SemanticSimilarityEvaluator, RelevancyEvaluator, FaithfulnessEvaluator

evaluator_c = CorrectnessEvaluator(llm=ollama_llm)
evaluator_s = SemanticSimilarityEvaluator(embed_model=embed_model)
evaluator_r = RelevancyEvaluator(llm=ollama_llm)
evaluator_f = FaithfulnessEvaluator(llm=ollama_llm)

建立query_engine

當你呼叫 as_query_engine 時傳入 llm 和 embed_model，這會覆蓋掉（Override）當初在建立 qdrant_index 時設定的全域或索引層級的物件。

In [ ]:
qdrant_query_engine = qdrant_index.as_query_engine(similarity_top_k=5,
                                                   response_mode="compact")

In [ ]:
from llama_index.core.evaluation.eval_utils import get_responses, get_results_df
from llama_index.core.evaluation import BatchEvalRunner

eval_qs = final_dataset.questions
qr_pairs = final_dataset.qr_pairs
ref_response_strs = [r for (_, r) in qr_pairs]

實驗

In [ ]:
pred_responses = get_responses(eval_qs[:2], qdrant_query_engine, show_progress=True)

In [ ]:
len(pred_responses[0].source_nodes)

In [ ]:
evaluator_dict = {
    "correctness": evaluator_c,
    "faithfulness": evaluator_f,
    "relevancy": evaluator_r,
    "similarity": evaluator_s
}

batch_runner = BatchEvalRunner(evaluator_dict, workers=2, show_progress=True)

In [ ]:
eval_results = await batch_runner.aevaluate_responses(
    eval_qs[:2], # 問題 
    responses=pred_responses, # 回應 
    reference=ref_response_strs[:2] # `標準答案`
    
)

In [ ]:
results_df = get_results_df(
    [eval_results],
    names=["Qdrant"],
    metric_keys=['correctness', 'faithfulness', 'relevancy', 'similarity']
)
display(results_df)

In [ ]:
eval_results["correctness"]

## 基於關聯的節點解析器 (Relation-Based Node Parsers)

### HierarchicalNodeParser

此節點解析器會將節點切分成 **階層式節點（hierarchical nodes）**。  
也就是說，單一輸入會被切分成多個不同大小的階層，每個節點都會包含對其 **父節點** 的參考。

當與 `AutoMergingRetriever` 結合使用時，這可以讓我們在大多數子節點被檢索到時，自動將檢索到的節點替換為其父節點。  
這個過程可以為 LLM 提供更完整的上下文，有助於生成更精確的回答。

#### 參考資料
- [知乎專欄文章](https://zhuanlan.zhihu.com/p/678698001)

In [ ]:
rom llama_index.core.node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

## HierarchicalNodeParser + AutoMergingRetriever 範例



In [ ]:
from pathlib import Path

from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()

docs = loader.load("08物理.pdf")

In [ ]:
# docs0

In [ ]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs])
docs = [Document(text=doc_text)]

In [ ]:
from llama_index.core.node_parser import HierarchicalNodeParser

In [ ]:
node_parser = HierarchicalNodeParser.from_defaults()

In [ ]:
nodes = node_parser.get_nodes_from_documents(docs)

In [ ]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

In [ ]:
leaf_nodes = get_leaf_nodes(nodes)
root_nodes = get_root_nodes(nodes)

In [ ]:
len(nodes)

In [ ]:
len(root_nodes)

In [ ]:
len(leaf_nodes)

In [ ]:
leaf_nodes[0]

In [ ]:
root_nodes[0]

### Load into Storage
我們首先定義一個 **docstore**，將所有節點（nodes）加載進去。

接著，我們定義一個 **VectorStoreIndex**，只包含 **葉節點（leaf-level nodes）**。

In [ ]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads https://huggingface.co/BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
    embed_model=embed_model
)

### Define Retriever

In [ ]:
from llama_index.core.retrievers import AutoMergingRetriever

In [ ]:
base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [ ]:
query_str = "原子與原子核的組成"
nodes = retriever.retrieve(query_str)
base_nodes = base_retriever.retrieve(query_str)

In [ ]:
nodes

In [ ]:
base_nodes

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.openai import OpenAI

from initialization import credential_init

credential_init

llm = OpenAI(
    model="gpt-4o-mini",
)

In [ ]:
query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)
base_query_engine = RetrieverQueryEngine.from_args(base_retriever, llm=llm)

In [ ]:
response = query_engine.query(query_str)

In [ ]:
base_response = base_query_engine.query(query_str)

### 評估 (Evaluation)

我們將以更 **量化的方式** 評估階層式檢索器（hierarchical retriever）相較於基準檢索器（baseline retriever）的效果。

> ⚠️ 注意：這個過程可能花費較高成本。請小心使用，並根據預算調整樣本數量。

In [ ]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
import nest_asyncio

"""
nest_asyncio 會 打補丁（patch） 現有的事件循環，使得 在已經運行的事件循環中可以再次嵌套運行 asyncio。

換句話說，它讓你在 Jupyter Notebook 裡可以順利執行：

asyncio.run()

自己寫的 async 函數

各種需要事件循環的庫（例如 LLM SDK、WebSocket、FastAPI 的測試等）

而不會碰到「事件循環已經運行」的錯誤。
"""
nest_asyncio.apply()

In [ ]:
dataset_generator = DatasetGenerator(
    root_nodes[:5],
    llm=llm,
    show_progress=True,
    num_questions_per_chunk=3,
)

In [ ]:
eval_dataset = await dataset_generator.agenerate_dataset_from_nodes(num=15)

In [ ]:
eval_dataset.queries

In [ ]:
eval_dataset.responses

In [ ]:
root_nodes[0]

In [ ]:
root_nodes[4]

In [ ]:
eval_dataset.save_json("物理_eval_qr_dataset.json")

In [ ]:
# optional
# eval_dataset = QueryResponseDataset.from_json(
#     "物理_eval_qr_dataset.json"
# )

In [ ]:
from collections import defaultdict

import pandas as pd
from llama_index.core.evaluation import (
    CorrectnessEvaluator,
    SemanticSimilarityEvaluator,
    RelevancyEvaluator,
    FaithfulnessEvaluator,
    PairwiseComparisonEvaluator,
)


# NOTE: can uncomment other evaluators
evaluator_c = CorrectnessEvaluator(llm=llm)
# evaluator_s = SemanticSimilarityEvaluator(llm=eval_llm)
evaluator_r = RelevancyEvaluator(llm=llm)
evaluator_f = FaithfulnessEvaluator(llm=llm)

In [ ]:
from llama_index.core.evaluation import RelevancyEvaluator
from llama_index.llms.openai import OpenAI

"""
Despite the name, RelevancyEvaluator in current LlamaIndex versions is primarily answering:

“Is the response relevant to the query given the provided context?”
"""

evaluator = RelevancyEvaluator(llm=llm)

result = evaluator.evaluate(
    query="What is photosynthesis?",
    response="Photosynthesis is the process by which plants convert sunlight into energy.",
    contexts = [
    "Photosynthesis is a biological process where plants use sunlight to synthesize food from carbon dioxide and water."
]
)

print(result.score)

In [ ]:
from llama_index.core.evaluation.eval_utils import (
    get_responses,
    get_results_df,
)
from llama_index.core.evaluation import BatchEvalRunner

In [ ]:
eval_qs = eval_dataset.questions
qr_pairs = eval_dataset.qr_pairs
ref_response_strs = [r for (_, r) in qr_pairs]

In [ ]:
eval_qs

In [ ]:
qr_pairs

In [ ]:
pred_responses = get_responses(eval_qs, query_engine, show_progress=True)

In [ ]:
base_pred_responses = get_responses(
    eval_qs, base_query_engine, show_progress=True
)

In [ ]:
pred_response_strs = [str(p) for p in pred_responses]
base_pred_response_strs = [str(p) for p in base_pred_responses]

In [ ]:
str(pred_responses[0])

In [ ]:
evaluator_dict = {
    "correctness": evaluator_c,
    "faithfulness": evaluator_f,
    "relevancy": evaluator_r,
}
batch_runner = BatchEvalRunner(evaluator_dict, workers=2, show_progress=True)

In [ ]:
eval_results = await batch_runner.aevaluate_responses(
    eval_qs, responses=pred_responses, reference=ref_response_strs
)

In [ ]:
base_eval_results = await batch_runner.aevaluate_responses(
    eval_qs, responses=base_pred_responses, reference=ref_response_strs
)

In [ ]:
results_df = get_results_df(
    [eval_results, base_eval_results],
    ["Auto Merging Retriever", "Base Retriever"],
    ["correctness", "relevancy"],
)
display(results_df)

In [ ]:
ref_response_strs

In [ ]:
for base_eval_result, eval_result, reference_result in zip(base_eval_results['correctness'], eval_results['correctness'], ref_response_strs):
    if base_eval_result.score > eval_result.score:
        print("\n******")
        print(f"query: {base_eval_result.query}")
        print(f"reference: {reference_result}")
        print("\n")
        print(f"base_eval_result: {base_eval_result.response}, {base_eval_result.score}")
        print("\n")
        print(f"eval_result: {eval_result.response}, {eval_result.score}")
        print("\n******")